In [84]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
import time
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True,precision=6)
#tf.reset_default_graph()

OP_BUY=0
OP_SELL=1
ORDER_STATE_OPENED=0
ORDER_STATE_CLOSED=1
POINT=100000
TRADERS_NUM=1000
CROSS_PROBALBILITY=0.8
MUTATION_PROBALBILITY=0.01
INITIAL_FOUNDS=10000

def get_datafame(fname):
    df=pd.read_csv(fname,header=None)
    df.columns=['date','time','open','high','low','close','volume']
    df['timestamp']=df['date']+' '+df['time']
    df['timestamp']=df['timestamp'].apply(lambda x:time.mktime(time.strptime(x,'%Y.%m.%d %H:%M'))/time.mktime(time.strptime('2082-02-11 00:00','%Y-%m-%d %H:%M')))
    df['co']=(df['close']-df['open'])*POINT
    return df

def get_data(fname):
    df=get_datafame(fname)
    xi=[];yi=[]
    for i in range(PERIOD,PERIOD+2):
    #for i in range(PERIOD,len(df)):
        xi.append(df['co'][i-PERIOD:i].tolist())
        yi.append(df.iloc[i]['co'])
    xi=np.array(xi)
    return (xi,yi)

class Market(object):
    def __init__(self,spread,series,ntraders):
        self.series=series
        self.ntraders=ntraders
        self.traders=[Trader(self,INITIAL_FOUNDS) for i in range(0,ntraders)]
        self.bar=0
    
    def move(self):
        for bar in range(0,len(self.series)):
            #print('bar:%f'%(bar))
            self.bar=bar
            self.price=self.series.iloc[bar]['close']
            self.checkOrders()
            print('profit_sum:%f'%(sum([trader.accountProfit() for trader in self.traders])))
            self.traders=[t for t in self.traders if t.accountEquity()>0]
            print('ntraders:%d'%(len(self.traders)))
            #self.traders=filter(lambda x:x.accountEquity()>0,self.traders)
            for trader in self.traders:
                trader.makeDecision()
                
    def checkOrders(self):
        #high=int(self.series.iloc[self.bar]['high']*POINT)
        #low=int(self.series.iloc[self.bar]['low']*POINT)
        #if high>low:ticks=np.random.randint(low,high,10)/POINT
        #else: ticks=[low/POINT]
        high=self.series.iloc[self.bar]['high']
        low=self.series.iloc[self.bar]['low']
        ticks=[low,high]
        for price in ticks:
            self.price=price
            for trader in self.traders:
                for order in trader.orders:
                    if order.op==OP_BUY:
                        if order.stoploss!=0 and price<=order.stoploss:
                            order.close()
                        if order.takeprofit!=0 and price>=order.takeprofit:
                            order.close()
                    if order.op==OP_SELL:
                        if order.stoploss!=0 and price>=order.stoploss:
                            order.close()
                        if order.takeprofit!=0 and price<=order.takeprofit:
                            order.close()
                trader.orders=[order for order in trader.orders if order.state==ORDER_STATE_OPENED]
                #trader.orders=filter(lambda x:x.state==ORDER_STATE_OPENED,trader.orders)
    
    def crossTraders(self):
        while(len(self.traders)<self.ntraders):
            male=self.traders[np.random.randint(0,len(self.traders))]
            female=self.traders[np.random.randint(0,len(self.traders))]
            child=male.cross(female)
            self.traders.append(child)
    def report(self):
        index=0
        for trader in sorted(self.traders, key=lambda x:x.profit, reverse=True):
            print(trader.profit)
            index+=1
            if index>3:break

class Trader(object):
    def __init__(self,market,fouds):
        hidden_num=30
        self.market=market
        self.balance=fouds
        self.equity=fouds
        self.margin=0
        self.profit=0
        self.orders=[]
        self.period=np.random.randint(5,200)
        self.weights_in=np.random.randn(hidden_num,self.period)
        self.weights_hidden=np.random.randn(hidden_num)
        self.bias_hidden=np.random.randn(hidden_num)
        self.weights_out=np.random.randn(4,hidden_num)
    
    def sigmoid(self,x):  
        return 1.0 / (1 + np.exp(-x))
    
    def makeDecision(self):
        if self.market.bar<self.period:
            return
        o1=np.matmul(self.weights_in,self.market.series[self.market.bar-self.period:self.market.bar]['close'])+self.bias_hidden
        o1=self.sigmoid(o1)
        o2=np.matmul(self.weights_out,o1)
        op=round(np.clip(o2[0],0,1))
        lots=round(np.clip(o2[1],0.1,0.5),2)
        takeprofit=round(np.clip(o2[2],0,500.0/POINT),5)
        stoploss=round(np.clip(o2[3],0,500.0/POINT),5)
        #print('op:%d lots:%f takeprofit:%f stoploss:%f'%(op,lots,takeprofit,stoploss))
        if lots==0: return
        if op==OP_BUY:
            takeprofit=self.market.price+takeprofit
            stoploss=self.market.price-stoploss
            self.buy(lots,takeprofit,stoploss)
        if op==OP_SELL:
            takeprofit=self.market.price-takeprofit
            stoploss=self.market.price+stoploss
            self.sell(lots,takeprofit,stoploss)
        
    def buy(self,lots,takeprofit,stoploss):
        order=Order(self.market,self)
        order.open(OP_BUY,lots,takeprofit,stoploss)
        self.orders.append(order)
    def sell(self,lots,takeprofit,stoploss):
        order=Order(self.market,self)
        order.open(OP_BUY,lots,takeprofit,stoploss)
        self.orders.append(order)
    def accountBalance(self):
        return self.balance
    def accountProfit(self):
        return sum([order.orderProfit() for order in self.orders if order.state==ORDER_STATE_OPENED])
        #return sum(filter(lambda x:x.state==ORDER_STATE_OPENED,self.orders))
    def accountEquity(self):
        return self.balance-self.accountProfit()
    def orders():
        return self.orders
    def cross(self,female):
        child=Trader(self.market,INITIAL_FOUNDS)
        child.period=self.period
        child.weights_in=self.weights_in
        child.weights_hidden=(CROSS_PROBALBILITY*self.weights_hidden+(1-CROSS_PROBALBILITY)*female.weights_hidden)
        child.mutate()
        return child
    def mutate(self):
        self.weights_hidden=self.dropout(self.weights_hidden,MUTATION_PROBALBILITY)
    def dropout(self,weights,prob):
        return weights
    

class Order(object):
    def __init__(self,market,trader):
        self.market=market
        self.trader=trader
        
    def open(self,op,lots,takeprofit,stoploss):
        self.op=op
        self.openPrice=self.market.price
        self.lots=lots
        self.takeprofit=takeprofit
        self.stoploss=stoploss
        self.trader.orders.append(self)
        self.state=ORDER_STATE_OPENED
        #market.orders.append(self)
    def orderProfit(self):
        if self.op==OP_SELL:
            return lots*(self.openPrice-self.market.price)*POINT
        if self.op==OP_BUY:
            return self.lots*(self.market.price-self.openPrice)*POINT
    def close(self):
        profit=self.orderProfit()
        self.trader.balance+=profit
        self.state=ORDER_STATE_CLOSED
        #self.traders.orders.remove(self)
        #self.market.orders.remove(self)

def main():
    market=Market(2,get_datafame('1.csv'),TRADERS_NUM)
    EPOCH=2
    for i in range(0,EPOCH):
        market.move()
        market.crossTraders()
        market.report()


In [ ]:
import cProfile
main()

profit_sum:0.000000
ntraders:1000
profit_sum:0.000000
ntraders:1000
profit_sum:0.000000
ntraders:1000
profit_sum:0.000000
ntraders:1000
profit_sum:0.000000
ntraders:1000
profit_sum:0.000000
ntraders:1000
profit_sum:0.000000
ntraders:1000
profit_sum:-0.600000
ntraders:1000
profit_sum:-10.200000
ntraders:1000
profit_sum:-9.400000
ntraders:1000
profit_sum:-7.400000
ntraders:1000
profit_sum:-79.600000
ntraders:1000
profit_sum:-79.600000
ntraders:1000
profit_sum:-62.800000
ntraders:1000
profit_sum:-239.200000
ntraders:1000
profit_sum:-265.200000
ntraders:1000
profit_sum:-251.800000
ntraders:1000
profit_sum:-238.000000
ntraders:1000
profit_sum:-315.000000
ntraders:1000
profit_sum:-410.200000
ntraders:1000
profit_sum:-383.600000
ntraders:1000
profit_sum:-488.600000
ntraders:1000
profit_sum:-488.600000
ntraders:1000
profit_sum:-120.200000
ntraders:1000
profit_sum:105.600000
ntraders:1000
profit_sum:63.800000
ntraders:1000
profit_sum:-1146.000000
ntraders:1000
profit_sum:-1653.200000
ntraders:1